In [99]:
!pip install ctransformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached ctransformers-0.2.27-py3-none-any.whl.metadata (17 kB)
Using cached ctransformers-0.2.27-py3-none-any.whl (9.9 MB)


In [31]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [32]:
PINECONE_API_KEY="6f043c11-9675-47ee-b7ab-24161ed91eee"
PINECONE_API_ENV="aws-us-east-1"

In [33]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [34]:
extracted_data = load_pdf("data/")

In [7]:
# extracted_data

In [35]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [36]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [ ]:
#Creating Embeddings for Each of The Text Chunks & storing
#docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [37]:
for i, t in zip(range(len(text_chunks)), text_chunks):
   query_result = embeddings.embed_query(t.page_content)
   index.upsert(
   vectors=[
        {
            "id": str(i),  # Convert i to a string
            "values": query_result, 
            "metadata": {"text":str(text_chunk[i].page_content)} # meta data as dic
        }
    ],
    namespace="real" 
)

In [38]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [39]:
embeddings = download_hugging_face_embeddings()

In [40]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [41]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [42]:
# query_result

In [43]:
#Initializing the Pinecone
import os
from pinecone import Pinecone, ServerlessSpec
from langchain.vectorstores import Pinecone as PC
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
pc = Pinecone(
    api_key=os.environ.get("PINECONE_API_KEY")
)

index_name="medical-chatbot"
index = pc.Index(index_name)

#Creating Embeddings for Each of The Text Chunks & storing
#docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [76]:

# Define your query text
query_text = "What are Allergies?"

# Embed the query text
query_embedding = embeddings.embed_documents([query_text])[0]

search_results = index.query(
    namespace="real",
    vector=query_embedding,
    top_k=3,
    include_values=True,
    include_metadata=True
)

# Print the search results
for i, result in enumerate(search_results['matches']):
    print(f"Result {i+1}:")
    print(f"ID: {result['id']}")
    print(f"Score: {result['score']}")
    print(f"Text: {result['metadata']['text']}\n")



Result 1:
ID: 1437
Score: 0.695547938
Text: the itchy, scratchy nose, eyes, and throat common inallergic rhinitis .
The particular allergens to which a person is sensi-

Result 2:
ID: 1372
Score: 0.694514155
Text: GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies
Allergic rhinitis is commonly triggered by
exposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.
The presence of an allergen causes the
body's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.
IgE molecules attach to mast
cells, which contain histamine.HistaminePollen grains
Lymphocyte
FIRST EXPOSURE

Result 3:
ID: 1355
Score: 0.678831041
Text: allergens are the following:
• plant pollens
• animal fur and dander
• body parts from house mites (microscopic creatures
found in all houses)
• house dust• mold spores• cigarette smoke• solvents• cleaners
Common food allergens include the foll

In [87]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'real': {'vector_count': 7020}},
 'total_vector_count': 7020}

In [95]:


import os
import pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone


# initialize the vector store object
vectorstore = Pinecone(
    index, embeddings.embed_query, "text"
)


# Define your query text
query_text = "What are Allergies?"

# Perform the similarity search using the Pinecone index
search_results = vectorstore.similarity_search(query_text, k=3,namespace='real')


# Print the search results using Langchain's interface
print("Results using Langchain's interface to Pinecone index:")

for i, result in enumerate(search_results):
    print(f"Result {i+1}:")
    print(f"Page Content: {result.page_content}")
    print("\n")


Results using Langchain's interface to Pinecone index:
Result 1:
Page Content: the itchy, scratchy nose, eyes, and throat common inallergic rhinitis .
The particular allergens to which a person is sensi-


Result 2:
Page Content: GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies
Allergic rhinitis is commonly triggered by
exposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.
The presence of an allergen causes the
body's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.
IgE molecules attach to mast
cells, which contain histamine.HistaminePollen grains
Lymphocyte
FIRST EXPOSURE


Result 3:
Page Content: allergens are the following:
• plant pollens
• animal fur and dander
• body parts from house mites (microscopic creatures
found in all houses)
• house dust• mold spores• cigarette smoke• solvents• cleaners
Common food allergens include the followi

In [96]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [97]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [101]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q8_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [102]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

Input Prompt: What are allergies


/Users/wafavellaniyil/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Response :  Allergies occur when your body's immune system overreacts to a substance that is normally harmless, such as pollen, dust mites, or certain foods. This overreaction can cause a range of symptoms, including sneezing, runny nose, itchy eyes, and difficulty breathing. In severe cases, allergies can lead to anaphylaxis, a life-threatening reaction that requires immediate medical attention. There is no cure for allergies, but there are several treatments available to help manage symptoms and prevent reactions.


Input Prompt: What is Acne


Response :  Acne is a common skin condition characterized by inflammation of the oil glands in the skin, which can lead to the formation of pimples, blemishes, and other imperfections. There are several factors that can contribute to the development of acne, including hormonal changes, stress, and genetics. Acne can be treated with a variety of methods, including topical creams and oral medications, as well as lifestyle changes such as regular exercise and a healthy diet. It is important to note that acne can have emotional and psychological impacts on individuals, and seeking support from mental health professionals may be beneficial.


Input Prompt: What is migraine


Response :  Migraines are a type of headache that can cause severe, debilitating pain, often accompanied by other symptoms such as sensitivity to light and sound, nausea, and vomiting. They can last for hours or even days and can significantly impact a person's quality of life. Migraines are believed to be caused by changes in blood flow and neurotransmitter levels in the brain, although the exact cause is still not fully understood. There are several different types of migraines, including common migraines, complex migraines, and ocular migraines. Treatment options for migraines include medications, lifestyle changes, and alternative therapies such as acupuncture and herbal supplements.
